In [6]:
#%%capture output

# once finished, try on all HAPI records and export output to file by doing the top and bottom comments

from datetime import datetime, timedelta
import Scripts
from Scripts import create_sqlite_database, execution, executionALL
from hapiclient import hapi
from time import sleep

# check and see if data can be retrieved from HAPI server
def DataChecker(server, dataset, start, stop, parameters, attempts):
    # control loop
    dataFound = False
    
    # place in a try/except to wait x seconds and try again if this fails
    try:
        # Get data
        #sleep(5.0)
        data, meta = hapi(server, dataset, parameters, start, stop)

    # if error arises
    # needs to be specific error, not blank
    except HAPIError:
        print("Error raised, trying again")

    # if no error arises
    else:
        dataLen = len(data)
        print("The length of data is " + str(dataLen))

        # search for data
        while not dataFound:
            for entry in data:
                #print(type(data[0][1]))
                
                # once data is found, end loop
                if str(data[0][0]) != "":
                    dataFound = True
                    print("Data was successfully accessed")
                    print("Example data looks like " + str(data[0][0]))
                    break
    finally:
        attempts += 1
        return dataFound, attempts

# retrieve prodKeys associated with HAPI URLs

# input abs path of database file you wish to query from
conn = create_sqlite_database("/home/jovyan/HDRL-Internship-2024/SPASE_Data_20240716.db")

# fails data checks 
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 9"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 10"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 15"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 24"""

HapiStmt = """ SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 15"""
prodKeys = []
prodKeys = execution(HapiStmt, conn)
#prodKeys = ['HELIOS1_E6_1HOUR_PARTICLE_FLUX']
#print("The prodKeys are " + str(prodKeys))


server = 'https://cdaweb.gsfc.nasa.gov/hapi'
#dataset    = 'HELIOS1_E6_1HOUR_PARTICLE_FLUX'
#start      = '2011-08-06T00:00:00'
#stop       = '2011-08-06T00:30:00'
#parameters = 'SE_LAT'

# iterate thru prodKeys
for prodKey in prodKeys:
    # check if multiple prodKeys for same URL (mult keys in one string)
    if ", " in prodKey:
        print("This HAPI URL has multiple product keys.")
        index = prodKeys.index(prodKey)
        prodKey = prodKey.replace("\'", "")
        # keep separating them until each prodKey is in own string
        # while ", " in prodKey:
        before, sep, after = prodKey.partition(", ")
        prodKeys[index] = before # remove this line if need to check all keys
        #prodKeys[index] = after
        #prodKeys.insert(index, before)
        prodKey = prodKeys[index]
    
    # control loop
    dataFound = False
    # list that holds all parameters for a given server
    paramNames =  []

    dataset = str(prodKey)

    # retrieve all parameters and the start date from the server
    sleep(5.0)
    meta = hapi(server,dataset)
    # get parameters
    for k, v in meta.items():
        if k == "parameters":
            for params in v:
                for key, value in params.items():
                    if key == "name":
                        paramNames.append(value)
        # get start date
        elif k == "startDate":
            start = v

    #print("Start time is " + start)
    #print("Stop time is " + stop)
    #print(paramNames)
    
    # count of attempts to get data
    attempts = 0
    # dictionary that holds datetime obj for each interval
    intervals = {}
    intervals["10s"] = ""
    intervals["10min"] = ""
    intervals["1hr"] = ""
    intervals["1d"] = ""
    intervals["3d"] = ""
    intervals["1w"] = ""
    intervals["1mon"] = ""
    
    # create incremental intervals to test for data check
    date, sep, time = start.partition("T")
    time = time.replace("Z", "")
    dt_string = date + " " + time
    dt_obj = datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S")
    # seconds
    intervals["10s"] = dt_obj + timedelta(seconds=10)
    # minutes
    intervals["10min"] = dt_obj + timedelta(minutes=10)
    # hour
    intervals["1hr"] = dt_obj + timedelta(hours=1)
    # day
    intervals["1d"] = dt_obj + timedelta(days=1)
    # 3 days
    intervals["3d"] = dt_obj + timedelta(days=3)
    # week
    intervals["1w"] = dt_obj + timedelta(weeks=1)
    # month
    intervals["1mon"] = dt_obj + timedelta(weeks=4,days=2)

    # to iterate thru all parameters in a server, use the for loop below to enclose the code below
    # for parameters in paramNames[1:]:
    
    # UNFINISHED: have data check occur in increasingly larger start/stop intervals until data is returned
    #              and only check for one parameter (Time is fine)
    parameter = paramNames[0]
    for k, v in intervals.items():
        if not dataFound:
            stop = str(v)
            stop = stop.replace(" ", "T") + "Z"
            print("Checking parameter " + str(parameter) + " in HAPI record with id " + dataset +
                  " at the interval of " + str(k))
            dataFound, attempts = DataChecker(server, dataset, start, stop, parameter, attempts)
    # if all intervals fail -> no data
    if not dataFound:
    # inputs "HAPI info check passed after 1 attempt. HAPI data check failed after 7 attempts."
    #     into "Error" column in TestResults associated with that HAPI URL
        print("No data was found")
        break

#with open("../DatalinkCheckOutput.txt", "w") as file:
 #   file.write(output.stdout)

Checking parameter Time in HAPI record with id IM_K0_EUV at the interval of 10s
The length of data is 1
Data was successfully accessed
Example data looks like b'2000-03-28T09:56:32.269Z'
Checking parameter Time in HAPI record with id IM_K0_SIE at the interval of 10s
The length of data is 1
Data was successfully accessed
Example data looks like b'2000-04-25T09:52:03.525Z'
Checking parameter Time in HAPI record with id IM_K0_SIP at the interval of 10s
The length of data is 1
Data was successfully accessed
Example data looks like b'2000-04-25T09:41:42.869Z'
Checking parameter Time in HAPI record with id IM_K0_WIC at the interval of 10s
The length of data is 1
Data was successfully accessed
Example data looks like b'2000-04-25T07:44:03.660Z'
Checking parameter Time in HAPI record with id IM_K0_HENA at the interval of 10s
The length of data is 1
Data was successfully accessed
Example data looks like b'2000-04-21T01:50:04.360Z'
Checking parameter Time in HAPI record with id IM_K0_LENA at the

In [ ]:
print("The program is done!")

In [ ]:
# goes in else block bw print and break
HAPIErrorStmt = f""" UPDATE TestResults
                                SET Errors = 'HAPI info check passed after 1 attempt. HAPI data check \
                                failed after 7 attempts.'
                                FROM (SELECT SPASE_id, prodKey FROM TestResults
                                        INNER JOIN MetadataEntries USING (SPASE_id))
                                WHERE prodKey = '{dataset}' """
                Record_id = execution(f""" SELECT rowNum 
                                        FROM (SELECT TestResults.rowNum, SPASE_id, prodKey FROM TestResults 
                                            INNER JOIN MetadataEntries USING (SPASE_id))
                                        WHERE prodKey = '{dataset}';""", conn)
                executionALL(HAPIErrorStmt, conn)
                print(f"Sent error message to a TestResults entry with the row number {Record_id}")

In [ ]:
# call .py file directly from notebook
#%run ./HAPICheck.py > ../DatalinkCheckOutput.txt